In [1]:
from utils import build_url, save_json

API_KEY_FILE = 'api_key.txt'
REGION = 'euw1'
BASE_URL = 'api.riotgames.com'

QUEUE_TYPE = 'RANKED_SOLO_5x5'

# Ligas de las que extraer jugadores
PIPELINE = [
    {
        'url': '/lol/league/v4/challengerleagues/by-queue/{queue}',
        'name': 'CHALLENGER',
        'completed': False,
        'params': {
            'queue': QUEUE_TYPE,
        }
    },
    {
        'url': '/lol/league/v4/grandmasterleagues/by-queue/{queue}',
        'name': 'GRANDMASTER',
        'completed': False,
        'params': {
            'queue': QUEUE_TYPE,
        }
        
    },
    {
        'url': '/lol/league/v4/masterleagues/by-queue/{queue}',
        'name': 'MASTER',
        'completed': False,
        'params': {
            'queue': QUEUE_TYPE,
        }
    },
    {
        'url': '/lol/league/v4/entries/{queue}/{tier}/{division}',
        'completed': False,
        'params': {
            'queue': QUEUE_TYPE,
            'tier': 'DIAMOND',
            'division': 'I'
        },
        'page': 1
    },
    {
        'url': '/lol/league/v4/entries/{queue}/{tier}/{division}',
        'completed': False,
        'params': {
            'queue': QUEUE_TYPE,
            'tier': 'DIAMOND',
            'division': 'II',
        },
        'page': 1
    },
    {
        'url': '/lol/league/v4/entries/{queue}/{tier}/{division}',
        'completed': False,
        'params': {
            'queue': QUEUE_TYPE,
            'tier': 'DIAMOND',
            'division': 'III'
        },
        'page': 1
    },
    {
        'url': '/lol/league/v4/entries/{queue}/{tier}/{division}',
        'completed': False,
        'params': {
            'queue': QUEUE_TYPE,
            'tier': 'DIAMOND',
            'division': 'IV'
        },
        'page': 1
    }
]

with open(API_KEY_FILE) as f:
    api_key = f.read()

In [2]:
import requests, copy, time
import os, shutil, pickle

execution_cache_dir = 'current_execution'
leagues_state_file = f'{execution_cache_dir}/remaining_leagues.pkl'
players_file = f'{execution_cache_dir}/players.pkl'

headers = {
  'X-Riot-Token': api_key
}

os.makedirs(execution_cache_dir, exist_ok=True)
MIN_DELAY = 30/100 # esperar 30ms entre peticiones
delay = MIN_DELAY

# Leer un fichero del estado de la ejecución si existe o copiar PIPELINE
# para empezar desde 0
if os.path.exists(leagues_state_file) and os.stat(leagues_state_file).st_size != 0:
    with open(leagues_state_file, 'rb') as f:
        leagues_current_execution = pickle.load(f)
else:
    leagues_current_execution = copy.deepcopy(PIPELINE)

if os.path.exists(players_file) and os.stat(players_file).st_size != 0:
    with open(leagues_state_file, 'rb') as f:
        players = pickle.load(f)
else:
    players = []

# Filtrar las fases que ya estén completas
leagues_current_execution = list(filter(lambda league: not league['completed'], leagues_current_execution))

for league in leagues_current_execution:
    # Para las ligas paginadas, pedir en bucle hasta que no se devuelvan jugadores
    # Si alguna petición falla, aumentar delay
    # Si va bien, disminuir el delay hasta el inicial
    # Al terninar cada fase o página, guardar el estado
    if 'page' in league:
        params = league['params']
        while True:
            url = build_url(REGION, BASE_URL, league['url'].format(**league['params']))
            response = requests.get(url, headers=headers, params={'page': league['page']})
            if response.status_code == 200:
                players_temp = response.json()
                if not players_temp:
                    break
                players.extend(players_temp)
                print(f"{params['tier']} {params['division']} ({league['page']}): len: {len(players_temp)} / total: {len(players)}")
                delay = max(delay/2, MIN_DELAY)
                league['page'] += 1
            else:
                retry_after = int(response.headers['Retry-After'])
                delay += retry_after
                print(f"{params['tier']} {params['division']} ({league['page']}): Petición fallida: {response.text}")
                print(f'Esperando {retry_after}s')
            with open(leagues_state_file, 'wb') as f:
                pickle.dump(leagues_current_execution, f)
            with open(players_file, 'wb') as f:
                pickle.dump(players, f)
            time.sleep(delay)
    else:
        url = build_url(REGION, BASE_URL, league['url'].format(**league['params']))
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            players_temp = response.json()['entries']
            players.extend(players_temp)
            print(f"{league['name']}: len: {len(players_temp)} / total: {len(players)}")
            delay = max(delay/2, MIN_DELAY)
        else:
            retry_after = int(response.headers['Retry-After'])
            delay += retry_after
            print(f"{params['name']}: Petición fallida: {response.text}")
            print(f'Esperando {retry_after}s')
        time.sleep(delay)
    league['completed'] = True
    with open(leagues_state_file, 'wb') as f:
        pickle.dump(leagues_current_execution, f)
    with open(players_file, 'wb') as f:
        pickle.dump(players, f)

CHALLENGER: len: 300 / total: 300
GRANDMASTER: len: 700 / total: 1000
MASTER: len: 5134 / total: 6134
DIAMOND I (1): len: 205 / total: 6339
DIAMOND I (2): len: 205 / total: 6544
DIAMOND I (3): len: 205 / total: 6749
DIAMOND I (4): len: 205 / total: 6954
DIAMOND I (5): len: 205 / total: 7159
DIAMOND I (6): len: 205 / total: 7364
DIAMOND I (7): len: 205 / total: 7569
DIAMOND I (8): len: 205 / total: 7774
DIAMOND I (9): len: 205 / total: 7979
DIAMOND I (10): len: 205 / total: 8184
DIAMOND I (11): len: 205 / total: 8389
DIAMOND I (12): len: 205 / total: 8594
DIAMOND I (13): len: 205 / total: 8799
DIAMOND I (14): len: 205 / total: 9004
DIAMOND I (15): len: 205 / total: 9209
DIAMOND I (16): len: 205 / total: 9414
DIAMOND I (17): len: 205 / total: 9619
DIAMOND I (18): len: 205 / total: 9824
DIAMOND I (19): len: 205 / total: 10029
DIAMOND I (20): len: 205 / total: 10234
DIAMOND I (21): len: 205 / total: 10439
DIAMOND I (22): len: 205 / total: 10644
DIAMOND I (23): len: 205 / total: 10849
DIAMO

DIAMOND IV (48): len: 205 / total: 45452
DIAMOND IV (49): len: 205 / total: 45657
DIAMOND IV (50): len: 205 / total: 45862
DIAMOND IV (51): len: 205 / total: 46067
DIAMOND IV (52): len: 205 / total: 46272
DIAMOND IV (53): len: 205 / total: 46477
DIAMOND IV (54): len: 205 / total: 46682
DIAMOND IV (55): len: 205 / total: 46887
DIAMOND IV (56): len: 205 / total: 47092
DIAMOND IV (57): len: 205 / total: 47297
DIAMOND IV (58): len: 205 / total: 47502
DIAMOND IV (59): len: 205 / total: 47707
DIAMOND IV (60): len: 205 / total: 47912
DIAMOND IV (61): len: 205 / total: 48117
DIAMOND IV (62): len: 205 / total: 48322
DIAMOND IV (63): len: 205 / total: 48527
DIAMOND IV (64): len: 205 / total: 48732
DIAMOND IV (65): len: 205 / total: 48937
DIAMOND IV (66): len: 205 / total: 49142
DIAMOND IV (67): len: 205 / total: 49347
DIAMOND IV (68): len: 205 / total: 49552
DIAMOND IV (69): len: 205 / total: 49757
DIAMOND IV (70): len: 205 / total: 49962
DIAMOND IV (71): len: 205 / total: 50167
DIAMOND IV (72):

In [3]:
# Guardar JSON con los jugadores
fields = {'summonerId', 'summonerName'}
summoners = [{k: player[k] for k in fields} for player in players]
save_json(summoners, 'players.json')

In [4]:
# Limpiar estados
shutil.rmtree(execution_cache_dir)